## Install the package dependencies before running this notebook

In [31]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "/home/jmryan/cse151B/argo2/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

In [5]:
batch_sz = 4  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

In [ ]:
import matplotlib.pyplot as plt
import random


def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,:,0], out[i,:,1])

        
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    show_sample_batch(sample_batch)

/tmp/ipykernel_452/153573212.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [2]:
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')

In [3]:
ROOT_PATH = "/home/jmryan/cse151B/argo2/train"
x = pickle.load(open(ROOT_PATH + '/austin_inputs', 'rb'))
y = pickle.load(open(ROOT_PATH + '/austin_outputs', 'rb'))

In [4]:
m_x = pickle.load(open(ROOT_PATH + '/miami_inputs', 'rb'))
m_y = pickle.load(open(ROOT_PATH + '/miami_outputs', 'rb'))
p_x = pickle.load(open(ROOT_PATH + '/pittsburgh_inputs', 'rb'))
p_y = pickle.load(open(ROOT_PATH + '/pittsburgh_outputs', 'rb'))
d_x = pickle.load(open(ROOT_PATH + '/dearborn_inputs', 'rb'))
d_y = pickle.load(open(ROOT_PATH + '/dearborn_outputs', 'rb'))
w_x = pickle.load(open(ROOT_PATH + '/washington-dc_inputs', 'rb'))
w_y = pickle.load(open(ROOT_PATH + '/washington-dc_outputs', 'rb'))
pa_x = pickle.load(open(ROOT_PATH + '/palo-alto_inputs', 'rb'))
pa_y = pickle.load(open(ROOT_PATH + '/palo-alto_outputs', 'rb'))

In [5]:
x_all = [m_x,p_x,d_x,w_x,pa_x]
y_all = [m_y,p_y,d_y,w_y,pa_y]

In [8]:
x = torch.tensor(x, device=device)
y = torch.tensor(y, device=device)
new_y = torch.empty((2582460, 50, 2), device= device)
new_x =  torch.empty((2582460, 50, 2), device= device)
count = 0
for i in range(len(x)):
    feat = torch.cat([x[i],y[i]])
    for j in range(60):
        new_x[count] = feat[j:j+50,:]
        new_y[count] =  feat[j+1:j+51,:]
        count += 1


/tmp/ipykernel_99/2434146055.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, device=device)
/tmp/ipykernel_99/2434146055.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, device=device)


In [10]:
all_x = []
all_y = []
for k in range(len(x_all)):
    print(k)
    temp_x = torch.empty((len(x_all[k]) * 60, 50, 2), device=device)
    temp_y = torch.empty((len(x_all[k]) * 60, 50, 2), device=device)

    curr_x = torch.tensor(x_all[k],device=device)
    curr_y = torch.tensor(y_all[k], device=device)
    count = 0
    for i in range(len(curr_x)):
        feat = torch.cat([curr_x[i],curr_y[i]])
        for j in range(60):
            temp_x[count] = feat[j:j+50,:]
            temp_y[count] =  feat[j+1:j+51,:]
            count += 1
    all_x.append(temp_x)
    all_y.append(temp_y)

0
1
2
3
4


In [11]:
all_x.append(new_x)
all_y.append(new_y)

In [73]:
new_x.shape

torch.Size([2582460, 50, 2])

In [5]:
x = torch.tensor(x, device = device)
y = torch.tensor(y, device = device)

In [21]:
final = torch.empty((len(x),2), device=device)

In [22]:
len(torch.cat((x[0],y[0]), 0))

110

In [27]:
np.append(x[0],y[0])

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [23]:

for i in range(len(x)):
    final = torch.cat((final, torch.cat((x[i], y[i]), dim=0)))

In [24]:
final.shape

torch.Size([4777551, 2])

In [47]:
class LSTM(torch.nn.Module):
    def __init__(self, num_hidden, num_layers, input_size, seq_len, batch_size, device):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.seq_len = seq_len
        self.device = device
        self.num_hidden = num_hidden
        self.proj_size = 2
        self.batch_size = batch_size
        
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=num_hidden, num_layers=num_layers, batch_first=True, device=device)
#         self.fc = torch.nn.Linear(hidden_size, 128, device=device)
       # self.lstm2 = torch.nn.LSTM(input_size=input_size, hidden_size=num_hidden, num_layers=num_layers, batch_first=True, device=device)

        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(in_features = num_hidden, out_features = 100, device=device)
        self.h  = torch.rand(self.num_layers, batch_size, self.num_hidden, device=device)
        self.c = torch.rand(self.num_layers, batch_size, self.num_hidden, device=device)
    
    def forward(self,x):
        # Propagate input through LSTM
        h  = torch.rand(self.num_layers, self.batch_size, self.num_hidden, device=device)
        c = torch.rand(self.num_layers, self.batch_size, self.num_hidden, device=device)
        #h2  = torch.rand(self.num_layers, self.batch_size, self.num_hidden, device=device)
        #c2 = torch.rand(self.num_layers, self.batch_size, self.num_hidden, device=device)
        output, (h, c) = self.lstm(x, (h, c)) #lstm with input, hidden, and internal state
        #output, (h,c) = self.lstm2(h, (h2,c2))
        output = self.linear(h)
        output = torch.reshape(output, (self.batch_size, self.seq_len, self.input_size))
#         hn = hn.view(-1, self.num_hidden) #reshaping the data for Dense layer next
      #  //out = self.relu(hn) 
     #   //out = self.relu(out) #relu
    #out = self.fc(out) #Final Output
        return output

In [48]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    for i in range(0, num_examples, batch_size):
        batch_indices = np.array(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [49]:
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')

num_epochs = 100 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 2 #number of features
hidden_size = 128 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

#number of output classes 
lstm = LSTM(hidden_size, num_layers, input_size, 50, 1000, device)
torch.autograd.set_detect_anomaly(True)

In [50]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 

In [51]:
for epoch in range(num_epochs):
    for i in range(len(all_x)):
        
        t = data_iter(1000, all_x[i], all_y[i])
        count = 1
        for x,y in t:
            if x.shape[0] != 1000:
                continue
            outputs = lstm.forward(
                x.float()) #forward pass
            optimizer.zero_grad() #caluclate the gradient, manually setting to 0
          # obtain the loss function
                #print(outputs[:,:,-1])
                #print(outputs.shape)
        #         print(y.float())
        #         print(y.float().shape)
            loss = criterion(outputs[:,-1,:], y.float()[:,-1,:])
                #print(loss)
            loss.backward(retain_graph=True) #calculates the loss of the loss function

            optimizer.step() #improve from loss, i.e backprop
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 1550038.62500
Epoch: 1, loss: 1227097.25000
Epoch: 2, loss: 916522.68750
Epoch: 3, loss: 770832.25000
Epoch: 4, loss: 701812.43750
Epoch: 5, loss: 559282.75000
Epoch: 6, loss: 455944.09375
Epoch: 7, loss: 342184.37500
Epoch: 8, loss: 300084.87500
Epoch: 9, loss: 230136.81250
Epoch: 10, loss: 196574.59375
Epoch: 11, loss: 173345.21875
Epoch: 12, loss: 131042.29688
Epoch: 13, loss: 105493.14062
Epoch: 14, loss: 117499.24219
Epoch: 15, loss: 85170.40625
Epoch: 16, loss: 137158.85938
Epoch: 17, loss: 108051.41406
Epoch: 18, loss: 162010.64062
Epoch: 19, loss: 80562.24219
Epoch: 20, loss: 73112.44531
Epoch: 21, loss: 192089.32812
Epoch: 22, loss: 101979.99219
Epoch: 23, loss: 122745.36719
Epoch: 24, loss: 73207.00781
Epoch: 25, loss: 82827.96875
Epoch: 26, loss: 62664.28125
Epoch: 27, loss: 85124.80469
Epoch: 28, loss: 74182.28125
Epoch: 29, loss: 46632.05078
Epoch: 30, loss: 31574.59375
Epoch: 31, loss: 57600.67188
Epoch: 32, loss: 78185.71875
Epoch: 33, loss: 63404.18750
E

In [86]:
m,b = next(t)

In [66]:
m

tensor([[[-2.0193e+02,  1.1567e+03],
         [-2.0180e+02,  1.1567e+03],
         [-2.0167e+02,  1.1567e+03],
         [-2.0154e+02,  1.1566e+03],
         [-2.0141e+02,  1.1566e+03],
         [-2.0129e+02,  1.1566e+03],
         [-2.0116e+02,  1.1565e+03],
         [-2.0104e+02,  1.1565e+03],
         [-2.0092e+02,  1.1565e+03],
         [-2.0079e+02,  1.1564e+03],
         [-2.0067e+02,  1.1564e+03],
         [-2.0055e+02,  1.1564e+03],
         [-2.0043e+02,  1.1563e+03],
         [-2.0031e+02,  1.1563e+03],
         [-2.0019e+02,  1.1562e+03],
         [-2.0007e+02,  1.1562e+03],
         [-1.9995e+02,  1.1562e+03],
         [-1.9983e+02,  1.1561e+03],
         [-1.9971e+02,  1.1561e+03],
         [-1.9959e+02,  1.1560e+03],
         [-1.9947e+02,  1.1560e+03],
         [-1.9934e+02,  1.1560e+03],
         [-1.9921e+02,  1.1560e+03],
         [-1.9908e+02,  1.1559e+03],
         [-1.9894e+02,  1.1559e+03],
         [-1.9879e+02,  1.1559e+03],
         [-1.9864e+02,  1.1558e+03],
 

In [65]:
m[:,-1,:]

tensor([[ -195.6793,  1155.2272],
        [  229.8162,  -769.5313],
        [ -578.6198,    30.5431],
        [ 3426.3870, -1784.4888]], device='cuda:0')

In [ ]:
plt.scatter(x[i][:,:][:,0], x[i][:,:][:,1], c='blue', s=1)
plt.scatter(y[i][:,:][:,0], y[i][:,:][:,1], c='red', s=1)
plt.show()

In [15]:
device

device(type='cuda')